# Importing Libraries 

In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.ensemble import IsolationForest
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score
import seaborn as sns 
from sklearn.preprocessing import StandardScaler
from pyod.models.abod import ABOD
from pyod.utils.data import evaluate_print
from pyod.utils.example import visualize
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support

# Data importing and Pre-processing 

In [15]:
df=pd.read_csv("/Users/saikarthik/Desktop/Nexus-S1 Deakin /data.csv")

In [16]:
df.head()


,timestamp,value
0,2013-12-02 21:15:00,73.967322
1,2013-12-02 21:20:00,74.935882
2,2013-12-02 21:25:00,76.124162
3,2013-12-02 21:30:00,78.140707
4,2013-12-02 21:35:00,79.329836


In [17]:
df.tail()

,timestamp,value
22690,2014-02-19 15:05:00,98.185415
22691,2014-02-19 15:10:00,97.804168
22692,2014-02-19 15:15:00,97.135468
22693,2014-02-19 15:20:00,98.056852
22694,2014-02-19 15:25:00,96.903861


In [18]:
df.shape

(22695, 2)

In [19]:
df.isnull().sum()

timestamp    0
value        0
dtype: int64

In [20]:
data_converted = pd.concat([df[['timestamp']], df.drop('timestamp', axis=1).astype('float32')], axis=1)


In [21]:
df=data_converted

In [22]:
df.to_csv("cleaned.csv")

# Exploratory Data Analysis

In [ ]:
mean_value = df['value'].mean()
median_value = df['value'].median()
std_dev_value = df['value'].std()

mean_value, median_value, std_dev_value


In [ ]:
sns.histplot(df["value"])

In [ ]:
plt.figure(figsize=(12, 6))
sns.lineplot(x='timestamp', y='value', data=df)
plt.title('Value over Time')
plt.xlabel('Timestamp')
plt.ylabel('Value')
plt.show()